# Lecture - Rerun.io

This notebook is a lecture of Mining Massive Dataset. 

For a lecture on usefully python packages & how to use them.

I decided to pick rerun.io

**Writtern By:** Guy Arieli

**Date:** April 2025

## Dependencies

In [1]:
!pip3 install "matplotlib>=3.7" nuscenes-devkit rerun-sdk

#### Imports

In [2]:
import rerun as rr
import rerun.utilities as rr_utils
from export_gps import derive_latlon
import rerun.blueprint as rrb
from typing import Any, Final
import matplotlib
import numpy as np
import time
import nuscenes
import pathlib
import math 
import warnings

warnings.filterwarnings("ignore")

## Introduction

**mutli-model:**
 integration and analysis of multiple types of data—such as text, images, audio, video, and sensor data—within a single system or model.

### What is Rerun?

Rerun is building the multimodal data stack to model, ingest, store, query and view robotics-style data. It's used in areas like robotics, spatial and embodied AI, generative media, industrial processing, simulation, security, and health.

![alt text](https://static.rerun.io/rerun-overview-new/1752fc259eef34f3aa8151b21b5937bc0bc2ad38/full.png)


1. Use the Rerun SDK to log multimodal data from your code or load it from storage
2. View live or recorded data in the standalone viewer or embedded in your app
3. Build layouts and customize visualizations interactively in the UI or through the SDK
4. Query recordings to get clean dataframes into tools like Pandas, Polars, or DuckDB
5. Extend Rerun when you need to

## So What Are Gonna Learn ? 

We'll start from a simpile visualization of car in the word. 

**Firstly,** Create simple static car in intersection 

**Secondary,** Create simple dynamic car in intersection with some movment. However the movement will not be prefect and this tool for analysing and seeing the problem in real time 

**Thirdly,** And most interstingly we'll Visualize the [nuScenes dataset](https://www.nuscenes.org/) including lidar, radar, images, and bounding boxes data.

#### Static Cars

In [3]:
class Color:
    Red = (255, 0, 0, 255)
    Gray = (128, 128, 128, 255)

In [4]:
def static_car_location(
    name: str = "multi-car-scene-static",
    padding: int = 5,
) -> None:
    rr.init(name, spawn=True)

    positions = np.array([
        [0, 0, 0],            # Center car
        [padding / 2, 0, 0],  # Right
        [-padding / 2, 0, 0], # Left
        [0, padding, 0],      # Top
        [0, -padding, 0],     # Bottom
    ], dtype=np.float32)

    colors = np.array([
        Color.Red,
        Color.Gray,
        Color.Gray,
        Color.Gray,
        Color.Gray
    ])

    half_sizes = np.tile([1.0, 2.0, 0.5], (5, 1))  # Repeat size for all cars
    labels = [f"Car {i}" for i in range(len(positions))]

    rr.log("cars", rr.Boxes3D(
        centers=positions,
        half_sizes=half_sizes,
        colors=colors,
        labels=labels
    ))

static_car_location()

In [ ]:
static_car_location(padding=10)

#### Dynamic Intersection

In [5]:
def animate_cars_different_speeds(
    name: str = "multi-car-scene-different-speeds",
    padding: int = 5,
    steps: int = 50,
    step_size: float = 0.1,
    delay: float = 0.05,
) -> None:
    rr.init(name, spawn=True)

    # Initial positions
    base_positions = np.array([
        [0, 0, 0],            # Center car
        [padding / 2, 0, 0],  # Right
        [-padding / 2, 0, 0], # Left
        [0, padding, 0],      # Top
        [0, -padding, 0],     # Bottom
    ], dtype=np.float32)

    colors = np.array([
        Color.Red,
        Color.Gray,
        Color.Gray,
        Color.Gray,
        Color.Gray
    ])

    speeds = np.array([-1.0, -0.5, -1.5, -0.2, -0.8])  #

    half_sizes = np.tile([1.0, 2.0, 0.5], (5, 1))
    labels = [f"Car {i}" for i in range(len(base_positions))]

    for step in range(steps):
        # Move each car in Y direction based on its own speed
        offsets = np.zeros_like(base_positions)
        offsets[:, 1] = step * step_size * speeds
        moved_positions = base_positions + offsets

        rr.set_time_sequence("frame", step)
        rr.log("cars", rr.Boxes3D(
            centers=moved_positions,
            half_sizes=half_sizes,
            colors=colors,
            labels=labels
        ))

        time.sleep(delay)

animate_cars_different_speeds()


#### Dynamic Intersection

This example demonstrates the ability to read and visualize scenes from the nuScenes dataset, which is a public large-scale dataset specifically designed for autonomous driving.
The scenes in this dataset encompass data collected from a comprehensive suite of sensors on autonomous vehicles.
These include 6 cameras, 1 LIDAR, 5 RADAR, GPS and IMU sensors.
Consequently, the dataset provides information about the vehicle's pose, the images captured, the recorded sensor data and the results of object detection at any given moment.

<picture>
  <img src="https://static.rerun.io/nuscenes_dataset/3724a84d6e95f15a71db2ccc443fb67bfae58843/full.png" alt="">
  <source media="(max-width: 480px)" srcset="https://static.rerun.io/nuscenes_dataset/3724a84d6e95f15a71db2ccc443fb67bfae58843/480w.png">
  <source media="(max-width: 768px)" srcset="https://static.rerun.io/nuscenes_dataset/3724a84d6e95f15a71db2ccc443fb67bfae58843/768w.png">
  <source media="(max-width: 1024px)" srcset="https://static.rerun.io/nuscenes_dataset/3724a84d6e95f15a71db2ccc443fb67bfae58843/1024w.png">
  <source media="(max-width: 1200px)" srcset="https://static.rerun.io/nuscenes_dataset/3724a84d6e95f15a71db2ccc443fb67bfae58843/1200w.png">
</picture>

#### Explore The Data

###### Const Values

In [6]:
DESCRIPTION = """
# nuScenes

Visualize the [nuScenes dataset](https://www.nuscenes.org/) including lidar, radar, images, and bounding boxes data.

The full source code for this example is available
[on GitHub](https://github.com/rerun-io/rerun/blob/latest/examples/python/nuscenes_dataset).
"""

DATASET_DIR: Final =  "nuscenes-data"

cmap = matplotlib.colormaps["turbo_r"]

norm = matplotlib.colors.Normalize(
    vmin=3.0,
    vmax=75.0,
) 

scene_name = "scene-0061"
dataset_version = "v1.0-mini"
seconds = float("inf")


##### Helper Functions

In [7]:
def nuscene_sensor_names(nusc: nuscenes.NuScenes, scene_name: str) -> list[str]:
    """Return all sensor names in the scene."""

    sensor_names = set()

    scene = next(s for s in nusc.scene if s["name"] == scene_name)
    first_sample = nusc.get("sample", scene["first_sample_token"])
    for sample_data_token in first_sample["data"].values():
        sample_data = nusc.get("sample_data", sample_data_token)
        if sample_data["sensor_modality"] == "camera":
            current_camera_token = sample_data_token
            while current_camera_token != "":
                sample_data = nusc.get("sample_data", current_camera_token)
                sensor_name = sample_data["channel"]
                sensor_names.add(sensor_name)
                current_camera_token = sample_data["next"]

    # For a known set of cameras, order the sensors in a circle.
    ordering = {
        "CAM_FRONT_LEFT": 0,
        "CAM_FRONT": 1,
        "CAM_FRONT_RIGHT": 2,
        "CAM_BACK_RIGHT": 3,
        "CAM_BACK": 4,
        "CAM_BACK_LEFT": 5,
    }
    return sorted(sensor_names, key=lambda sensor_name: ordering.get(sensor_name, float("inf")))


def setup_blueprint(nusc: nuscenes.NuScenes, scene_name: str) -> rrb.Blueprint:
    sensor_views = [
        rrb.Spatial2DView(
            name=sensor_name,
            origin=f"world/ego_vehicle/{sensor_name}",
            contents=["$origin/**", "world/anns"],
            overrides={"world/anns": rr.Boxes3D(fill_mode="majorwireframe")},
        )
        for sensor_name in nuscene_sensor_names(nusc, scene_name)
    ]
    return rrb.Blueprint(
        rrb.Vertical(
            rrb.Horizontal(
                rrb.Spatial3DView(
                    name="3D",
                    origin="world",
                    defaults=[rr.Pinhole(image_plane_distance=5.0)],
                    overrides={"world/anns": rr.Boxes3D(fill_mode="solid")},
                ),
                rrb.Vertical(
                    rrb.TextDocumentView(origin="description", name="Description"),
                    rrb.MapView(
                        origin="world",
                        name="MapView",
                        zoom=rrb.archetypes.MapZoom(18.0),
                        background=rrb.archetypes.MapBackground(rrb.components.MapProvider.OpenStreetMap),
                    ),
                    row_shares=[1, 1],
                ),
                column_shares=[3, 1],
            ),
            rrb.Grid(*sensor_views),
            row_shares=[4, 2],
        ),
        rrb.TimePanel(state="collapsed"),
    )


In [8]:
def add_description(description: str) -> None:
    rr.log(
        "description",
        rr.TextDocument(description, media_type=rr.MediaType.MARKDOWN),
        static=True,
    )

In [ ]:
def log_sensor_calibration(sample_data: dict[str, Any], nusc: nuscenes.NuScenes) -> None:
    """Log sensor calibration (pinhole camera, sensor poses, etc.)."""
    sensor_name = sample_data["channel"]
    calibrated_sensor_token = sample_data["calibrated_sensor_token"]
    calibrated_sensor = nusc.get("calibrated_sensor", calibrated_sensor_token)
    rotation_xyzw = np.roll(calibrated_sensor["rotation"], shift=-1)  # go from wxyz to xyzw
    rr.log(
        f"world/ego_vehicle/{sensor_name}",
        rr.Transform3D(
            translation=calibrated_sensor["translation"],
            rotation=rr.Quaternion(xyzw=rotation_xyzw),
            relation=rr.TransformRelation.ParentFromChild,
        ),
        static=True,
    )
    if len(calibrated_sensor["camera_intrinsic"]) != 0:
        rr.log(
            f"world/ego_vehicle/{sensor_name}",
            rr.Pinhole(
                image_from_camera=calibrated_sensor["camera_intrinsic"],
                width=sample_data["width"],
                height=sample_data["height"],
            ),
            static=True,
        )



def log_camera_position_while_extract_metadata(nusc: nuscenes.NuScenes, scene_name: str, max_time_sec: float) -> tuple:
    scene = next(s for s in nusc.scene if s["name"] == scene_name)

    location = nusc.get("log", scene["log_token"])["location"]

    rr.log("world", rr.ViewCoordinates.RIGHT_HAND_Z_UP, static=True)

    first_sample_token = scene["first_sample_token"]
    first_sample = nusc.get("sample", scene["first_sample_token"])

    first_lidar_token = ""
    first_radar_tokens = []
    first_camera_tokens = []
    for sample_data_token in first_sample["data"].values():
        sample_data = nusc.get("sample_data", sample_data_token)
        log_sensor_calibration(sample_data, nusc) # where each camera look in the car 
        if sample_data["sensor_modality"] == "lidar":
            first_lidar_token = sample_data_token
        elif sample_data["sensor_modality"] == "radar":
            first_radar_tokens.append(sample_data_token)
        elif sample_data["sensor_modality"] == "camera":
            first_camera_tokens.append(sample_data_token)
    
    first_timestamp_us = nusc.get("sample_data", first_lidar_token)["timestamp"]
    max_timestamp_us = first_timestamp_us + 1e6 * max_time_sec

    return first_sample_token, location, first_lidar_token, max_timestamp_us, first_camera_tokens, first_radar_tokens

In [10]:

def log_lidar_and_ego_pose(
    location: str,
    first_lidar_token: str,
    nusc: nuscenes.NuScenes,
    max_timestamp_us: float,
) -> None:
    current_lidar_token = first_lidar_token

    ego_trajectory_lat_lon = []

    while current_lidar_token != "":
        sample_data = nusc.get("sample_data", current_lidar_token)
        sensor_name = sample_data["channel"]

        if max_timestamp_us < sample_data["timestamp"]: # has finish run 
            break

        rr.set_time("timestamp", timestamp=sample_data["timestamp"] * 1e-6)

        ego_pose = nusc.get("ego_pose", sample_data["ego_pose_token"])
        rotation_xyzw = np.roll(ego_pose["rotation"], shift=-1)  # go from wxyz to xyzw
        position_lat_lon = derive_latlon(location, ego_pose)
        ego_trajectory_lat_lon.append(position_lat_lon)
        rr.log(        # add location in minimap
            "world/ego_vehicle",
            rr.Transform3D(
                translation=ego_pose["translation"],
                rotation=rr.Quaternion(xyzw=rotation_xyzw),
                axis_length=10.0,  # The length of the visualized axis.
                relation=rr.TransformRelation.ParentFromChild,
            ),
            rr.GeoPoints(lat_lon=position_lat_lon, radii=rr.Radius.ui_points(8.0), colors=0xFF0000FF),
        )
        rr.log( # add location trajectory in minimap
            "world/ego_vehicle/trajectory",
            rr.GeoLineStrings(lat_lon=ego_trajectory_lat_lon, radii=rr.Radius.ui_points(1.0), colors=0xFF0000FF),
        )
        # 
        current_lidar_token = sample_data["next"]
        # 
        data_file_path =pathlib.Path(nusc.dataroot) / sample_data["filename"]
        pointcloud = nuscenes.nuscenes.LidarPointCloud.from_file(str(data_file_path))
        points = pointcloud.points[:3].T  # shape after transposing: (num_points, 3)
        point_distances = np.linalg.norm(points, axis=1)
        point_colors = cmap(norm(point_distances))
        rr.log(f"world/ego_vehicle/{sensor_name}", rr.Points3D(points, colors=point_colors)) # all points in the word recived from camera

In [11]:
def log_cameras(first_camera_tokens: list[str], nusc: nuscenes.NuScenes, max_timestamp_us: float) -> None:
    """Log camera data."""
    for first_camera_token in first_camera_tokens:
        current_camera_token = first_camera_token
        while current_camera_token != "":
            sample_data = nusc.get("sample_data", current_camera_token)
            if max_timestamp_us < sample_data["timestamp"]:
                break
            sensor_name = sample_data["channel"]
            rr.set_time("timestamp", timestamp=sample_data["timestamp"] * 1e-6)
            data_file_path = pathlib.Path(nusc.dataroot) / sample_data["filename"]
            rr.log(f"world/ego_vehicle/{sensor_name}", rr.EncodedImage(path=data_file_path))
            current_camera_token = sample_data["next"]


In [12]:
def log_radars(first_radar_tokens: list[str], nusc: nuscenes.NuScenes, max_timestamp_us: float) -> None:
    """Log radar data."""
    for first_radar_token in first_radar_tokens:
        current_camera_token = first_radar_token
        while current_camera_token != "":
            sample_data = nusc.get("sample_data", current_camera_token)
            if max_timestamp_us < sample_data["timestamp"]:
                break
            sensor_name = sample_data["channel"]
            rr.set_time("timestamp", timestamp=sample_data["timestamp"] * 1e-6)
            data_file_path = pathlib.Path(nusc.dataroot) / sample_data["filename"]
            pointcloud = nuscenes.nuscenes.RadarPointCloud.from_file(str(data_file_path))
            points = pointcloud.points[:3].T  # shape after transposing: (num_points, 3)
            point_distances = np.linalg.norm(points, axis=1)
            point_colors = cmap(norm(point_distances))
            rr.log(
                f"world/ego_vehicle/{sensor_name}",
                rr.Points3D(points, colors=point_colors),
            )
            current_camera_token = sample_data["next"]

In [13]:
def log_annotations(location: str, first_sample_token: str, nusc: nuscenes.NuScenes, max_timestamp_us: float) -> None:
    """Log 3D bounding boxes."""
    label2id: dict[str, int] = {}
    current_sample_token = first_sample_token
    while current_sample_token != "":
        sample_data = nusc.get("sample", current_sample_token)
        if max_timestamp_us < sample_data["timestamp"]:
            break
        rr.set_time("timestamp", timestamp=sample_data["timestamp"] * 1e-6)
        ann_tokens = sample_data["anns"]
        sizes = []
        centers = []
        quaternions = []
        class_ids = []
        lat_lon = []
        for ann_token in ann_tokens:
            ann = nusc.get("sample_annotation", ann_token)

            rotation_xyzw = np.roll(ann["rotation"], shift=-1)  # go from wxyz to xyzw
            width, length, height = ann["size"]
            sizes.append((length, width, height))  # x, y, z sizes
            centers.append(ann["translation"])
            quaternions.append(rr.Quaternion(xyzw=rotation_xyzw))
            if ann["category_name"] not in label2id:
                label2id[ann["category_name"]] = len(label2id)
            class_ids.append(label2id[ann["category_name"]])
            lat_lon.append(derive_latlon(location, ann))

        rr.log(
            "world/anns",
            rr.Boxes3D(
                sizes=sizes,
                centers=centers,
                quaternions=quaternions,
                class_ids=class_ids,
            ), # create bounding box 
            rr.GeoPoints(lat_lon=lat_lon), # minimap location as geopoint
        )
        current_sample_token = sample_data["next"]

    annotation_context = [(i, label) for label, i in label2id.items()]
    rr.log("world/anns", rr.AnnotationContext(annotation_context), static=True) # add maping between number to text


##### Code

In [14]:
nusc = nuscenes.NuScenes(version=dataset_version, dataroot=DATASET_DIR, verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.261 seconds.
Reverse indexing ...
Done reverse indexing in 0.0 seconds.


In [15]:
rr.init("car-dataset-example", spawn=True)
blueprint = setup_blueprint(nusc, scene_name)
rr.send_blueprint(blueprint)
add_description(DESCRIPTION)

In [16]:
first_sample_token, location, first_lidar_token, max_timestamp_us, first_camera_tokens, first_radar_tokens = log_camera_position_while_extract_metadata(nusc, scene_name, max_time_sec=seconds)
first_sample_token, location, first_lidar_token, max_timestamp_us, first_camera_tokens, first_radar_tokens

('ca9a282c9e77460f8360f564131a8af5',
 'singapore-onenorth',
 '9d9bf11fb0e144c8b446d54a8a00184f',
 inf,
 ['e3d495d4ac534d54b321f50006683844',
  'aac7867ebf4f446395d29fbd60b63b3b',
  '79dbb4460a6b40f49f9c150cb118247e',
  '03bea5763f0f4722933508d5999c5fd8',
  '43893a033f9c46d4a51b5e08a67a1eb7',
  'fe5422747a7d4268a4b07fc396707b23'],
 ['37091c75b9704e0daa829ba56dfa0906',
  '11946c1461d14016a322916157da3c7d',
  '491209956ee3435a9ec173dad3aaf58b',
  '312aa38d0e3e4f01b3124c523e6f9776',
  '07b30d5eb6104e79be58eadf94382bc1'])

In [17]:
log_lidar_and_ego_pose(location, first_lidar_token, nusc, max_timestamp_us)

In [18]:
log_cameras(first_camera_tokens, nusc, max_timestamp_us)

In [19]:
log_radars(first_radar_tokens, nusc, max_timestamp_us)

In [20]:
log_annotations(location, first_sample_token, nusc, max_timestamp_us)